# UCI Epileptic Seizure Recognition Data Set

Link to the dataset: https://archive.ics.uci.edu/ml/datasets/Epileptic+Seizure+Recognition

In [425]:
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import transforms

In [426]:
class UCIEpilepsy(Dataset):
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path).iloc[:, 1:].to_numpy().astype(np.single)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):        
        data = torch.from_numpy(self.data[idx,2:])
        label = torch.from_numpy(self.data[idx,178:])
        return data, label

dataset = UCIEpilepsy(data_path='data/data.csv')
batch_size = 64
test_split = .2
shuffle_dataset = True
random_seed = 42
dataset_size = len(dataset)

# Creating data indices for training and validation splits:
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
test_indices, train_indices = indices[:split], indices[split:]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

In [427]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(177, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 5),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.linear_relu_stack(x)

In [428]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=177, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=5, bias=True)
    (5): ReLU()
  )
)


In [429]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [430]:
for epoch in range(2): # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels.squeeze(1).long()-1)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
print('Finished Training')

Finished Training
